# Single-Session vmPFC θ SFC & Hipp γ↔vmPFC θ ir-PAC
Working spec for a one-session pipeline that quantifies hippocampal spike-to-vmPFC theta phase locking and hippocampal gamma amplitude coupling to vmPFC theta phase.

## Notebook Roadmap
1. Configure session, storage paths, and analysis parameters.
2. Load NWB session data through a common adapter that surfaces canonical tables/arrays.
3. Preprocess LFPs (vmPFC θ phase, hippocampal γ amplitude) with optional CAR plus Hilbert transforms.
4. Build hippocampal unit × vmPFC channel pairs (hemisphere-aware).
5. Compute vmPFC θ spike-field coherence with spike-count balancing and jittered surrogate z-scores.
6. Compute hippocampal γ ↔ vmPFC θ inter-regional PAC with trial-count balancing, shuffle surrogates, and optional lag sweeps.
7. Aggregate per-pair metrics into session-level stats/visuals.
8. Emit QC figures, tables, and summaries into a structured output directory.
9. Append runtime/context metadata for reproducibility.

## Inputs & Interface Contract
All downstream analysis blocks expect the following canonical objects (regardless of whether they were pulled directly from NWB):

- **Events / Trials** – `trial_table`: DataFrame with `trial_id`, `condition_label`, `rt`, `maint_onset_time` (absolute seconds), `is_correct`, plus any auxiliary columns. Conditioning (e.g., `use_only_correct`, `conditions = ['L1','L3']`) is enforced during adapter loading.
- **Spikes** – `spike_times_by_unit`: dict mapping `unit_id → np.ndarray` of spike times (seconds). `unit_meta`: DataFrame with `unit_id`, anatomical `area` labels (e.g., `hipp`) and `hemisphere` tags, ideally with the source electrode/channel index.
- **LFP** – `lfp_by_channel`: dict `chan_id → np.ndarray`, along with `lfp_fs` (Hz), `lfp_start_time` (absolute seconds), and `channel_meta` describing each channel's area, hemisphere, bad-channel flags, and reference metadata. These support CAR/bipolar re-referencing before bandpass/Hilbert steps.
- **Session meta** – lightweight dict containing `session_id`, patient/subject identifiers, session date, and the reference scheme (if available).

The adapter defined in `nwb_analysis.cfc.prepare_session_structures` enforces this contract whether data are read from NWB, so the core analysis can focus on: (1) extracting [−0.5, +3.0] s windows around maintenance, (2) analyzing the [0, 2.5] s theta/gamma content, (3) equalizing spikes/trials per condition, and (4) computing surrogate-based z-scores (spike jitter for SFC, trial shuffle for PAC with optional lag sweeps).


## 1. Setup & Imports

In [1]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", context="talk")
warnings.filterwarnings("ignore", category=RuntimeWarning)

from nwb_analysis import get_subject_files
from nwb_analysis.cfc import (
    prepare_session_structures,
    compute_vmPFC_theta_phase,
    compute_hipp_gamma_envelope,
    compute_irpac,
    run_session_stats,
    plot_session_summary,
    save_session_outputs,
)


## 2. Config & Session Selection

In [ ]:
# Session + storage config
DATA_DIR = Path('/Users/jundazhu/SBCAT/000673')
SESSION_ID = 'SBCAT_Example_001'  # can be partial stem or left as example
SESSION_PATH_OVERRIDE = None      # Path('/absolute/path/to/sub-5_ses-1_ecephys+image.nwb') to skip globbing
OUT_DIR = Path('outputs')
OUT_DIR.mkdir(parents=True, exist_ok=True)
USE_ONLY_CORRECT = True

SESSION_OUT_DIR = OUT_DIR / SESSION_ID / 'sfc_irpac_single_session'
SESSION_OUT_DIR.mkdir(parents=True, exist_ok=True)

CONDITIONS = ['L1', 'L3']
EPOCH_ANALYZE = (0.0, 2.5)       # s window used for analyses relative to maintenance onset
EPOCH_LABEL = 'maintenance'
THETA_BAND = (3.0, 7.0)
PHASE_BAND_LABEL = 'theta'
GAMMA_BAND = (30.0, 140.0)
GAMMA_BAND_LABEL = 'gamma'
CAR_MODE = 'hemisphere'
MIN_TRIALS_PER_COND = 20
PAC_SURR_N = 500
LAG_GRID = np.arange(-0.150, 0.155, 0.005)  # seconds
EXCLUDE_LAG_S = (0.010, 0.020)              # ignore central lags when reporting lag-swept PAC
PEAK_TO_PEAK_THRESH = 3000.0  # µV; set to None to skip automatic rejection
PAC_ALPHA = 0.05

ANALYSIS_PARAMS = {
    'conditions': CONDITIONS,
    'epoch_analyze': EPOCH_ANALYZE,
    'epoch_label': EPOCH_LABEL,
    'theta_band': THETA_BAND,
    'phase_band_label': PHASE_BAND_LABEL,
    'gamma_band': GAMMA_BAND,
    'gamma_band_label': GAMMA_BAND_LABEL,
    'car_mode': CAR_MODE,
    'min_trials': MIN_TRIALS_PER_COND,
    'pac_surrogates': PAC_SURR_N,
    'lag_grid_s': LAG_GRID.tolist(),
    'exclude_lag_s': EXCLUDE_LAG_S,
    'max_peak_to_peak': PEAK_TO_PEAK_THRESH,
    'pac_alpha': PAC_ALPHA,
}

print(f"Session ID: {SESSION_ID}")
print(f"Outputs → {SESSION_OUT_DIR}")


Session ID: SBCAT_Example_001
Outputs → outputs/SBCAT_Example_001/sfc_irpac_single_session


### Resolve session file

In [3]:
from pathlib import Path


def resolve_session_path(session_id: str, data_dir: Path, path_override: Path | None = None) -> Path:
    '''Resolve an NWB file path using override, direct path, or globbing.'''
    candidates = []
    if path_override is not None:
        candidates.append(Path(path_override).expanduser())
    if session_id:
        candidates.append(Path(session_id).expanduser())
        candidates.append((data_dir / session_id).expanduser())
    for candidate in candidates:
        if candidate.suffix == '.nwb' and candidate.exists():
            return candidate
    if session_id:
        matches = sorted(data_dir.rglob(f"*{session_id}*.nwb"))
        if matches:
            return matches[0]
    files = get_subject_files(data_dir)
    if not files:
        raise FileNotFoundError(f"No NWB files found under {data_dir}")
    if session_id:
        print(f"[warn] Could not find '{session_id}' under {data_dir}. Falling back to {files[0].name}")
    return files[0]


SESSION_PATH = resolve_session_path(SESSION_ID, DATA_DIR, SESSION_PATH_OVERRIDE)
print(f"Using NWB file: {SESSION_PATH}")


[warn] Could not find 'SBCAT_Example_001' under /Users/jundazhu/SBCAT/000673. Falling back to sub-1_ses-1_ecephys+image.nwb
Using NWB file: /Users/jundazhu/SBCAT/000673/sub-1/sub-1_ses-1_ecephys+image.nwb


## 3. Load Data via Adapter

In [4]:
session_data = prepare_session_structures(
    session_id=SESSION_ID,
    session_path=SESSION_PATH,
    conditions=CONDITIONS,
    use_only_correct=USE_ONLY_CORRECT,
)

trial_table = session_data['trial_table']
spike_times_by_unit = session_data['spike_times_by_unit']
unit_meta = session_data['unit_meta']
lfp_by_channel = session_data['lfp_by_channel']
lfp_fs = session_data['lfp_fs']
lfp_start_time = session_data['lfp_start_time']
channel_meta = session_data['channel_meta']
session_meta = session_data['session_meta']

print(f"Trials: {len(trial_table)} | Units: {len(spike_times_by_unit)} | Channels: {len(lfp_by_channel)} @ {lfp_fs:.1f} Hz")
trial_table.head()


/Users/jundazhu/.pyenv/versions/3.10.17/lib/python3.10/site-packages/pynwb/ecephys.py:158: UserWarning: ElectricalSeries 'LFPs': The second dimension of data does not match the length of electrodes. Your data may be transposed.
  warnings.warn("%s '%s': The second dimension of data does not match the length of electrodes. "


Trials: 136 | Units: 46 | Channels: 70 @ 400.0 Hz


,condition_label,rt,maint_onset_time,is_correct,id,loads,PicIDs_Encoding1,PicIDs_Encoding2,PicIDs_Encoding3,PicIDs_Probe,...,timestamps_Encoding1_end,timestamps_Encoding2,timestamps_Encoding2_end,timestamps_Encoding3,timestamps_Encoding3_end,timestamps_Maintenance,timestamps_Probe,timestamps_Response,response_accuracy,probe_in_out
0,L3,1.660094,8.542684,True,0,3,201,101,501,501,...,4.203154,4.393779,6.401091,6.535497,8.542684,8.542684,11.746151,13.406245,1,1
1,L3,0.997281,21.168179,True,1,3,202,102,502,502,...,16.906399,16.947618,18.955961,19.159867,21.168179,21.168179,24.032771,25.030052,1,1
2,L3,0.819468,32.644268,True,2,3,301,401,103,301,...,28.265895,28.458301,30.466706,30.635925,32.644268,32.644268,35.436236,36.255704,1,1
3,L1,0.848031,39.653265,True,3,1,503,0,0,503,...,39.653265,0.000000,0.000000,0.000000,0.000000,39.653265,42.396920,43.244951,1,1
4,L3,2.271156,50.802136,True,4,3,402,203,504,103,...,46.591325,46.634856,48.643137,48.793824,50.802136,50.802136,53.367728,55.638884,1,0


## 4. Preprocessing (CAR, bandpass, Hilbert)

### Processing Details
**LFP (vmPFC reference):** Drop channels flagged as bad (metadata `bad` column) or exceeding the configurable peak-to-peak bound; band-pass (3–7 Hz) with zero-phase filtering and extract instantaneous phase via Hilbert. Hippocampal LFPs undergo analogous filtering in the γ bands (default 70–140 Hz, optional 30–55 Hz) with Hilbert amplitude envelopes for PAC.
**Spikes (hipp units):** For each trial's maintenance window ([−0.5, +3.0] s), collect spikes from hippocampal single units. SFC samples theta phase at each spike time, then equalizes spike counts across conditions by random subsampling to the shared minimum (repeated 200× to stabilize MVL estimates).
**Metrics & surrogates:**
- SFC uses the circular mean vector length (MVL = |(1/K)∑e^{iϕ}|). Surrogates jitter spikes within ±0.25 s of their trial to build μ/σ for z-scoring.
- ir-PAC uses amplitude-weighted MVL between vmPFC θ phase and hippocampal γ amplitude. Trials are shuffled (phase ↔ amp) within condition to generate surrogate distributions; optional lag sweeps shift amp vs phase across −150…+150 ms (excluding |lag| <10–20 ms) before computing MVL.


In [ ]:
theta_phase = compute_vmPFC_theta_phase(
    lfp_by_channel=lfp_by_channel,
    channel_meta=channel_meta,
    trial_table=trial_table,
    sampling_rate=lfp_fs,
    lfp_start_time=lfp_start_time,
    epoch_analyze=EPOCH_ANALYZE,
    theta_band=THETA_BAND,
    car_mode=CAR_MODE,
    trial_pad_sec=0.5,
    filter_order=4,
)

hipp_gamma_amp = compute_hipp_gamma_envelope(
    lfp_by_channel=lfp_by_channel,
    channel_meta=channel_meta,
    trial_table=trial_table,
    sampling_rate=lfp_fs,
    lfp_start_time=lfp_start_time,
    epoch_analyze=EPOCH_ANALYZE,
    gamma_band=GAMMA_BAND,
    trial_pad_sec=0.5,
    filter_order=4,
)

print(f"vmPFC θ channels: {len(theta_phase)}")
print(f"Hipp γ channels: {len(hipp_gamma_amp)}")

## 5. LFP Pair Formation (hipp γ channel × vmPFC θ channel)
Pairs enumerate hippocampal LFP channels supplying γ envelopes and vmPFC channels supplying θ phase. Channels must share a hemisphere when labels are available; otherwise they are paired globally.


In [6]:
hipp_channels = channel_meta[channel_meta['area'].str.contains('hipp', case=False, na=False)].copy()
vm_channels = channel_meta[channel_meta['area'].str.contains('vmpfc', case=False, na=False)].copy()
hipp_channels['hemisphere'] = hipp_channels.get('hemisphere', 'unknown').fillna('unknown')
vm_channels['hemisphere'] = vm_channels.get('hemisphere', 'unknown').fillna('unknown')

pairs = []
for hipp in hipp_channels.itertuples():
    for vm in vm_channels.itertuples():
        if hipp.hemisphere != 'unknown' and vm.hemisphere != 'unknown' and hipp.hemisphere != vm.hemisphere:
            continue
        pair_id = f"hipp{hipp.chan_id}_vm{vm.chan_id}"
        pairs.append({
            'pair_id': pair_id,
            'unit_id': hipp.chan_id,
            'hipp_chan_id': hipp.chan_id,
            'unit_channel': hipp.chan_id,
            'hipp_hemisphere': hipp.hemisphere,
            'chan_id': vm.chan_id,
            'vm_hemisphere': vm.hemisphere,
        })

pair_table = pd.DataFrame(pairs)
print(f"Candidate hipp×vmPFC channel pairs: {len(pair_table)}")
pair_table.head()


Candidate hipp×vmPFC channel pairs: 98


,pair_id,unit_id,hipp_chan_id,unit_channel,hipp_hemisphere,chan_id,vm_hemisphere
0,hipp22_vm67,22,22,22,L,67,L
1,hipp22_vm68,22,22,22,L,68,L
2,hipp22_vm69,22,22,22,L,69,L
3,hipp22_vm70,22,22,22,L,70,L
4,hipp22_vm71,22,22,22,L,71,L


## 7. Hipp γ ↔ vmPFC θ ir-PAC

In [ ]:
pac_results = compute_irpac(
    pair_table=pair_table[:10],
    trial_table=trial_table,
    theta_phase=theta_phase,
    gamma_envelope=hipp_gamma_amp,
    lfp_fs=lfp_fs,
    epoch_analyze=EPOCH_ANALYZE,
    conditions=CONDITIONS,
    min_trials=MIN_TRIALS_PER_COND,
    n_surrogates=PAC_SURR_N,
    lag_grid_s=None,
    exclude_lag_s=EXCLUDE_LAG_S,
    significance_alpha=PAC_ALPHA,
    phase_band=THETA_BAND,
    amp_band=GAMMA_BAND,
    phase_band_label=PHASE_BAND_LABEL,
    amp_band_label=GAMMA_BAND_LABEL,
    epoch_label=EPOCH_LABEL,
)

pac_results.head()

## 8. Session-Level Stats

In [ ]:
stats_summary = run_session_stats(
    sfc_results=None,
    pac_results=pac_results,
    conditions=CONDITIONS,
)

stats_summary


## 9. QC & Plots

In [ ]:
example_pair_id = None
summary_figs = plot_session_summary(
    sfc_results=None,
    pac_results=pac_results,
    stats_summary=stats_summary,
    conditions=CONDITIONS,
    output_dir=SESSION_OUT_DIR,
    dpi=150,
)
print(f"Session summary figs: {summary_figs}")


## 10. Save Artifacts

In [ ]:
artifacts = save_session_outputs(
    session_id=SESSION_ID,
    session_path=SESSION_PATH,
    output_dir=SESSION_OUT_DIR,
    pair_table=None,
    sfc_results=None,
    pac_results=pac_results,
    stats_summary=stats_summary,
    analysis_params=ANALYSIS_PARAMS,
)

artifacts


## 11. Appendix (runtime info)

In [ ]:
import json
import platform
import sys
from datetime import datetime

runtime_info = {
    'timestamp': datetime.utcnow().isoformat() + 'Z',
    'python': sys.version,
    'platform': platform.platform(),
    'session_meta': session_meta,
    'analysis_params': ANALYSIS_PARAMS,
}

print(json.dumps(runtime_info, indent=2))